# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [19]:
spaceship.dropna(inplace=True)
spaceship.drop_duplicates(inplace=True)

In [31]:
#spaceship.drop(columns=["PassengerId", "Name"], inplace=True)
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x : x.split('/')[0])
spaceship["VIP"] = spaceship["VIP"].apply(lambda x : 1 if x == True else 0)

spaceship = pd.get_dummies(spaceship, columns=["HomePlanet", "CryoSleep", "Cabin", "Destination"])
spaceship

,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,False,True,...,True,False,False,False,False,False,False,False,False,True
1,24.0,0,109.0,9.0,25.0,549.0,44.0,True,True,False,...,False,False,False,False,True,False,False,False,False,True
2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,False,True,...,False,False,False,False,False,False,False,False,False,True
3,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,16.0,0,303.0,70.0,151.0,565.0,2.0,True,True,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,False,True,...,False,False,False,False,False,False,False,True,False,False
8689,18.0,0,0.0,0.0,0.0,0.0,0.0,False,True,False,...,False,False,False,False,False,True,False,False,True,False
8690,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,True,False,...,False,False,False,False,False,True,False,False,False,True
8691,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,False,True,...,False,False,False,True,False,False,False,True,False,False


In [35]:
X = spaceship.drop(columns=["Transported"])
y = spaceship["Transported"]

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#numeric_values = spaceship[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]]
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

base_estimator = DecisionTreeClassifier(max_depth=5)

# Initialize the AdaBoostClassifier with the base estimator and set the algorithm to SAMME
model_reg = AdaBoostClassifier(estimator=base_estimator, algorithm='SAMME')

param_grid = {
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.01, 0.1, 1]  # AdaBoost parameters
}


model = GridSearchCV(model_reg, param_grid, cv=5)
model.fit(X_train, y_train)

print("best hyperparamètres :", model.best_params_)




best hyperparamètres : {'learning_rate': 0.1, 'n_estimators': 100}


- Evaluate your model

In [67]:
print("accuracy of the best model :", model.best_score_)

accuracy of the best model : 0.8005492354329563


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [83]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],        
"estimator__max_leaf_nodes": [int(x) for x in np.linspace(start = 500, stop = 3000, num = 10)],
"estimator__max_depth":[int(x) for x in np.linspace(10, 110, num = 11)]}

- Run Grid Search

In [ ]:
model = RandomizedSearchCV(estimator = model_reg, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)
model.fit(X_train, y_train)
print("Best hyperparamètres :", model.best_params_)

- Evaluate your model

In [ ]:
print("accuracy of the best model :", model.best_score_)
